## Example Notebook to demonstrate the simplicity of saving data in QKIT

### First import the QKIT store module

In [ ]:
import qkit
from qkit.storage import store

... some imports to create some random data ...

In [ ]:
## for random data
from numpy.random import rand
from numpy import linspace,arange
import time
## number of points
nop = 101

### Create a data file object
The file name will be ** prepended by a unique 6 character identifier (UUID) ** 
and the file saved in a standard path below
qkit.cfg['datadir']

In [ ]:
h5d = store.Data(name='NewFancyData',mode = "a")

** h5d ** is now an empty hdf5 file, holding only some qkit structure

#### How to get the file path:

In [ ]:
print(h5d.get_filepath())

#### Now use qviewkit to open and view the file
##### Hint: in a fully started qkit environment (qkit.start()) This can be done by qkit.fid.view("UUID")

### Comment added to the basic hdf folder

options
 * comment (mandatory)
 * folder='data' | 'analysis' (optional, default is "data") 

In [ ]:
h5d.add_comment("New data has been created ....")

### Now we get to the important stuff: 

 * first we have to create the objects which are later used to store the data
 * then we can add data 

#### A coordinate is a 1-dim vector with bias values, e.g. set current, frequencies, etc.

In [ ]:
# add_coordinate()    <- for measurement boundaries/steps
# options: name (mandatory)
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 
f_co = h5d.add_coordinate('frequency', unit = "Hz",   comment = "VNA frequency scan")
I_co = h5d.add_coordinate('current',   unit = "A",    comment = "magnetic field current")
P_co = h5d.add_coordinate('power',     unit = "dBm",  comment = "microwave power")

#### A value_vector ie is a 1-dim vector measured values

In [ ]:
# add_value_vector()    <- for measurement data
# options: name (mandatory)
#        : x = X  (optional) coordinate vector in x direction, default: None
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 

T_vec = h5d.add_value_vector('temperature', x = None, 
                             unit = "K", comment = "save temperature values") 
Tc_vec = h5d.add_value_vector('critical_temperature', x = I_co, 
                              unit = "K", folder='analysis' ,comment = "save temperature values")

#### A value_matrix is a 2-dim tuple with two cooardinates with measurement values

In [ ]:
# add_value_matrix()    <- for measurement data
# convention: the last coordiante should be the one with the fastest changes:
#             e.g.  for a VNA scan x= magnetic field y= transmission frequency
# 
# options: name (mandatory)
#        : x = X  (optional) coordinate vector in x direction, default: None
#        : y = Y  (mandatory) coordinate vector in y direction / fastest changes
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 

amp_mx = h5d.add_value_matrix('amplitude', x = I_co , y = f_co, 
                              unit = "V", comment = "magic data")
pha_mx = h5d.add_value_matrix('phase',     x = I_co , y = f_co, 
                              unit = "rad", comment = "more magic data!")


#### A value_box is a 3-dim tuple with three cooardinates with measurement values

In [ ]:
# add_value_box()    <- for measurement data
# options: name (mandatory)
#        : x = X  (optional) coordinate vector in x direction, default: None
#        : y = Y  (optional) coordinate vector in y direction
#        : z = Z  (mandatory) coordinate vector in y direction /  fastest changes
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 

amp_bx = h5d.add_value_box('amplitude', x = I_co , y = f_co, z= P_co, 
                           unit = "V", comment = "magic data")
pha_bx = h5d.add_value_box('phase',     x = I_co , y = f_co, z= P_co, 
                           unit = "rad", comment = "more magic data!")




#### For free text, like settings or commments there is a special dataset which can be appended to add data

In [ ]:
#string array 
#add_textlist()
#options: name (mandatory)
#       : comment = ""   (optional)
#       : folder="data"  (optional)
# use the append method to add the text
settings = h5d.add_textlist("settings",comment = "my settings")
settings.append(u"vnapower = 10dBm")
settings.append(u"fridge attenuation=50db\n data jumps like Van Halen.")


#### If one want to plot a arbitrary dataset vs another arbitrary dataset use a view

In [ ]:
### Add a view on Data:

TvsTc_view = h5d.add_view("f_vs_I", x = f_co, y = I_co)

TvsTc_view.add(x=T_vec,y=Tc_vec)

Laspect_view = h5d.add_view("locked_aspect", x = f_co, y = f_co, 
                            view_params={'aspect':1.0, 'bgcolor':(100,200,100)})



### Finally add some data to the file
There are two functions to do so:
  * append(data)  <- this is the most frequently used method, it does what it says.
  * add(data)     <- this is used to set a dataset of lenght N to the data with length N

lets make an example:

In [ ]:
# now we add the coordinate data to the file
fs = linspace(1e9,5e9,nop)
Is = linspace(0e-3,10e-3,nop)


f_co.add(fs)
I_co.add(Is)

for i in arange(nop):
    #time.sleep(10)
    amp = rand(nop)
    pha = rand(nop)
    amp_mx.append(amp)
    pha_mx.append(pha)
    T_vec.append(float(rand(1)))
    Tc_vec.append(float(rand(1)))


#### Close the file at the end

In [ ]:
h5d.close_file()